# SatCom Capacity Time-Series Forecasting with Amazon SageMaker Autopilot

### Contents

1. [Introduction](#introduction)
1. [Setup](#setup)
1. [Model Training](#training)
1. [Real-Time Predictions (Inference)](#realtime)


### 1. Introduction <a name='introduction'>

This notebook uses Amazon SageMaker Autopilot to train a time-series model and produce real time predictions against the trained model. At the top-level, customers provide a set of tabular historical data on S3 and make an API to train a model. Once the model has been trained, you can elect to produce prediction as a batch or via a real-time endpoint. This
notebook only provides Real Time Inference. For batch inference see [here](https://github.com/aws/amazon-sagemaker-examples/blob/main/autopilot/autopilot_time_series.ipynb).  </n></n>  As part of the training process, SageMaker Autopilot manages and runs multiple time series models concurrently. All of these models are combined into a single ensembled model which blends the candidate models in a ratio that minimizes forecast error. Customers are provided with metadata and models for the ensemble and all underlying candidate models too. SageMaker Autopilot orchestrates this entire process and provides several artifacts as a result.

These artifacts include: 
- backtest (holdout) forecasts per base model over multiple time windows,
- accuracy metrics per base model,
- backtest results and accuracy metrics for the ensembled model,
- a scaled explainability report displaying the importance of each covariate and static metadata feature.
- all model artifacts are provided as well on S3, which can be registered or use for batch/real-time inference

### 2. Setup <a name='setup'>

In [ ]:
# Update boto3 using this method, or your preferred method
!pip install --upgrade boto3 --quiet
!pip install --upgrade sagemaker --quiet

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep
import datetime

region = boto3.Session().region_name
session = sagemaker.Session()

# MODIFY HERE : 
# Modify the following values default_bucket to use a bucket of your choosing
# Training data is in 1 subfolder whilst sample Real Time Inference files for prediction in another
# training inputs need to be in a separate folder from training outputs
bucket = 'forecast-satcom-autopilot-capacity'
train_prefix = 'dataset/train'
train_output_prefix = 'dataset/train_output'

# The prediction file is a small subset of the training data, again without the target variable (mHz) 
# values for the most recent day(s)
# We can generate that subset prediction csv via the same satcom-timeseries-autopilot-gen-fxn lambda
# Use mode = 'inf' instead of 'train' in the env variables (and set the startday or startmonth later)
rtinf_prefix = 'dataset/rtinf'
rtinf_satcom_pred_file = "satcom-autopilot-cap_1723517598.csv"   # replace with your generated filename


role = get_execution_role()
print(role)

# This is the client we will use to interact with SageMaker Autopilot
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

IMPORTANT: When training a model, your input data can contain a mixture of covariate and static item metadata. Take care to create future-dated rows that extend to the end of your prediction horizon. In the future-dated rows, carry your static item metadata and expected covariate values. Future-dated target-value (y) should be empty. You can observe the data programmatically or in a text editor as an example.

The structure of the CSV file provided is as follows:
- timestamp (required, TimestampAttributeName)
- airpressure (covariate)
- beam (required: ItemIdentifierAttributeName)
- dayofweek (covariate)
- hourofday (covariate)
- mHz (required: TargetAttributeName)

### 3. Model Training <a name='training'>

Establish an AutoML training job name

In [ ]:
timestamp_suffix = strftime("%Y%m%d-%H%M%S", gmtime())
auto_ml_job_name = "ts-" + timestamp_suffix
print("AutoMLJobName: " + auto_ml_job_name)

Define training job specifications. More information about [create_auto_ml_job_v2](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_auto_ml_job_v2.html) can be found in our SageMaker documentation.</n></n>This JSON body leverages the built-in sample data schema. Please consult the documentation to understand how to alter the parameters for your unique schema.

In [ ]:
input_data_config = [
    {  'ChannelType': 'training',
            'ContentType': 'text/csv;header=present',
            'CompressionType': 'None',
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://{}/{}/'.format(bucket, train_prefix),
            }
        }
    }
]

output_data_config = {'S3OutputPath': 's3://{}/{}/'.format(bucket, train_output_prefix)}

optimizaton_metric_config = {'MetricName': 'AverageWeightedQuantileLoss'}

automl_problem_type_config ={
        'TimeSeriesForecastingJobConfig': {
            'ForecastFrequency': '10min',
            'ForecastHorizon': 144,
            'ForecastQuantiles': ['p50','p70','p90'],
            'TimeSeriesConfig': {
                'TargetAttributeName': 'mHz',
                'TimestampAttributeName': 'timestamp',
                'ItemIdentifierAttributeName': 'beam',
            }
        }
    }

automl_train_tag=[
    {
        'Key': 'Name',
        'Value': 'satcom_forecast_train'
    }
]


With parameters now defined, invoke the [training job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_auto_ml_job_v2.html) and monitor for its completion. You can expect the training to take about 60-90 minutes.

In [ ]:
sm.create_auto_ml_job_v2(
    AutoMLJobName=auto_ml_job_name,
    AutoMLJobInputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLProblemTypeConfig = automl_problem_type_config,
    AutoMLJobObjective=optimizaton_metric_config,
    RoleArn=role,
    Tags=automl_train_tag
)

Next, we demonstrate a looping mechanism to query (monitor) job status. When the status is ```Completed```, you may review the accuracy of the model and decide whether to perform inference on a batch or real-time API basis as described in this notebook. Please consult documentation for [describe_auto_ml_job_v2](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/describe_auto_ml_job_v2.html) as needed.

In [ ]:
describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
       datetime.datetime.now(), describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(180)

Once training is completed, you can use the describe function to iterate over model leaderboard results. Below is an example to use the best candidate in the subsequent inference phase. Please consult our documentation on [create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html) as needed.

In [ ]:
best_candidate = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_containers = best_candidate['InferenceContainers'] 
best_candidate_name = best_candidate['CandidateName']

response = sm.create_model(
   ModelName = best_candidate_name,
   ExecutionRoleArn = role,
   Containers = best_candidate_containers
)

print('BestCandidateName:',best_candidate_name)
print('BestCandidateContainers:',best_candidate_containers)

### 4. Real-Time Predictions (Inference) <a name='realtime'>

If you want to perform real-time inference, review this section. 

Define a model, endpoint configuration and endpoint name using the candidate metadata. Consult [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_endpoint_config.html) documentation for more detail. Additionally, please adjust the ```InstanceType``` and ```InitialInstanceCount``` according to need.

IMPORTANT: The data you supply for inference must have at least four valid historical values for each time-series.

In [ ]:
endpoint_config_name = f"epc-{best_candidate_name}"
endpoint_name = f"ep-{best_candidate_name}"

endpoint_tag=[
    {
        'Key': 'Name',
        'Value': 'satcom_forecast_endpoint'
    }
]

endpoint_config_tag=[
    {
        'Key': 'Name',
        'Value': 'satcom_forecast_endpoint_config'
    }
]

production_variants = [
        {
            "InstanceType": "ml.m5.2xlarge",
            "InitialInstanceCount": 1,
            "ModelName": best_candidate_name,
            "VariantName": "AllTraffic",
        }
    ]

epc_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=production_variants,
    Tags=endpoint_config_tag
)

Next, you can deploy a real-time endpoint using the [create_endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_endpoint.html
) API. See the documentation for more details and options.

In [ ]:
sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name,
    Tags=endpoint_tag
)

Poll for endpoint to become ready to serve (InService)

In [ ]:
describe_response = sm.describe_endpoint(EndpointName=endpoint_name)

job_run_status = describe_response["EndpointStatus"]

while job_run_status not in ("Failed", "InService", "Stopped"):
    describe_response = sm.describe_endpoint(EndpointName=endpoint_name)
    job_run_status = describe_response["EndpointStatus"]

    print(
       datetime.datetime.now(), describe_response["EndpointStatus"])
    sleep(60)

The next cells help demonstrate opening a CSV file from S3 for inference. Alternately, this data could come from a database query or live application. In this example, the data is loaded into a Python memory object.

In [ ]:
# A small sample file that corresponds to the sample training dataset and trained model schema

rt_satcom_pred = "s3://" + bucket + "/" + rtinf_prefix + "/" + rtinf_satcom_pred_file
print(rt_satcom_pred)

#!aws s3 cp s3://amazon-forecast-samples/autopilot/real-time-payload.csv ./real-time-payload.csv
!aws s3 cp {rt_satcom_pred} ./{rtinf_satcom_pred_file}

In [ ]:
#input_file = './real-time-payload.csv'
input_file = f"{'./'}{rtinf_satcom_pred_file}"
print(input_file)
f=open(input_file,'r')
inference_data = f.read()
f.close()

# print first few chars to sanity check inference_data input for RT prediction
print (inference_data[0:100])

Method to instantiate SageMaker runtime client and [invoke endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime/client/invoke_endpoint.html). Please note the guidance given in the documentation page which says the response must be within 60 seconds of invocation. Response times are a function of payload size. You should take care to provide history for single time-series, and carefully testing the ability to perform predictions for more than one time-series at a time.

In [ ]:
sm_client = boto3.client('sagemaker-runtime')

In [ ]:
response = sm_client.invoke_endpoint(
    EndpointName= endpoint_name,
    Body= inference_data,
    ContentType = 'text/csv')

prediction = response['Body'].read().decode()

At this point, the results from the real-time API call are loaded into a variable called ```prediction```. You have several options on what you do with the results. A few are given here.

Optional: Example of saving the resulting real-time predictions to a local filesystem. Carefully plan your naming convention.

In [ ]:
dt_1 = datetime.datetime.now()
dt_secs = int(dt_1.timestamp())
print(dt_secs)

output_file = f"{'rt-satcom-pred-output'}_{dt_secs}{'.csv'}"
print(output_file)

f=open(output_file,'w')
f.write(prediction)
f.close()

Optional: Example of saving the resulting real-time predictions to a S3 object. Carefully plan your naming convention.

In [ ]:
#output_file = 'real-time-prediction-output.csv'
key='{}/results/{}'.format(rtinf_prefix, output_file)

print("SageMaker Autopilot Real Time Inference result. Bucket: " + bucket + ". Data: " + key)

In [ ]:
s3_client = boto3.client('s3')
s3_client.put_object(Body=prediction, Bucket=bucket, Key=key)

#### Cleanup Real-time Endpoint Resources

As needed, you can stop the endpoint and related billing costs as follows. When you need the endpoint again, you can follow the deployment steps again. Ideally, at a future time, another newer model is trained and able to be deployed as well.

In [ ]:
# sm.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

Caution: Do not delete the model if you intend on testing batch transformation too. If you do delete the model, you may redeploy it as long as the model artifact exists on S3. The next cell provides the container and S3 location for your best candidate model.

In [ ]:
# sm.delete_model(ModelName=best_candidate_name)